In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\teck1\\OneDrive\\Documents\\GitHub\\ai'

In [4]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path
    checkpoint_filepath_str: str

In [1]:
from DetectAnalytics.constants import *
from DetectAnalytics.utils.common import read_yaml, create_directories
import tensorflow as tf

In [11]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.preparing_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(model_ckpt_dir), Path(config.tensorboard_root_log_dir)])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath),
            checkpoint_filepath_str=config.checkpoint_model_filepath
        )
        
        return prepare_callback_config
    




    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.preparing_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "fighting_video")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE
        )

        return training_config

In [6]:
import time

In [7]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_filepath_str,
            save_best_only=True
        )
    
    @property
    def _create_earlystopping_callbacks(self):
        return tf.keras.callbacks.EarlyStopping(
            monitor = 'val_accuracy',
            patience = 10,
            restore_best_weights = True
        )
    
    @property
    def _create_reducelr_callbacks(self):
        return tf.keras.callbacks.ReduceLROnPlateau(
            monitor = 'val_loss',
            factor = 0.6,
            patience = 5,
            min_lr = 0.00001,
            verbose = 1
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks,
            self._create_earlystopping_callbacks,
            self._create_reducelr_callbacks
        ]

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
from sklearn.model_selection import train_test_split
import numpy as np

In [9]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def data_generator(self):
        features = np.load("features.npy")
        labels = np.load("labels.npy")
        video_files_paths = np.load("video_files_paths.npy")

        encoded_labels = tf.keras.utils.to_categorical(labels)

        self.features_train, self.features_test, self.labels_train, self.labels_test = train_test_split(
            features, 
            encoded_labels, 
            test_size = 0.1, 
            shuffle = True, 
            random_state = 42
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self, callback_list: list):

        self.model.fit(
            x = self.features_train,
            y = self.labels_train,
            epochs = self.config.params_epochs,
            batch_size = self.config.params_batch_size,
            shuffle = True,
            validation_split = 0.2,
            callbacks = callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [12]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.data_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2023-12-18 01:51:45,249: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-18 01:51:45,252: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-18 01:51:45,253: INFO: common: created directory at: artifacts]
[2023-12-18 01:51:45,255: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-12-18 01:51:45,256: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2023-12-18 01:51:45,258: INFO: common: created directory at: artifacts\training]
Epoch 1/50
180/180 [==============================] - ETA: 0s - loss: 0.6965 - accuracy: 0.4965

c:\Users\teck1\miniconda3\envs\detectai\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


180/180 [==============================] - 55s 286ms/step - loss: 0.6965 - accuracy: 0.4965 - val_loss: 0.6975 - val_accuracy: 0.4556 - lr: 0.0100
Epoch 2/50
180/180 [==============================] - 51s 285ms/step - loss: 0.6977 - accuracy: 0.4979 - val_loss: 0.6914 - val_accuracy: 0.4889 - lr: 0.0100
Epoch 3/50
180/180 [==============================] - 50s 279ms/step - loss: 0.6920 - accuracy: 0.5111 - val_loss: 0.6894 - val_accuracy: 0.5111 - lr: 0.0100
Epoch 4/50
180/180 [==============================] - 50s 278ms/step - loss: 0.6838 - accuracy: 0.5660 - val_loss: 0.6776 - val_accuracy: 0.5611 - lr: 0.0100
Epoch 5/50
180/180 [==============================] - 50s 276ms/step - loss: 0.6568 - accuracy: 0.6104 - val_loss: 0.6456 - val_accuracy: 0.6500 - lr: 0.0100
Epoch 6/50
180/180 [==============================] - 51s 283ms/step - loss: 0.6420 - accuracy: 0.6479 - val_loss: 0.6360 - val_accuracy: 0.6306 - lr: 0.0100
Epoch 7/50
180/180 [==============================] - 51s 281ms